This notebook use fstring you will need py36 or above

In [54]:
import logging

# BYOL: Error handling in python 
### 9 December 2019



> Inspired by Mario Corchero - Exceptional Exceptions PYCON 19
>
> https://youtu.be/V2fGAv2R5j8

# Motivation
- Review of exceptions handling in python
- Better debugging, common pitfall
- Accessible to any python level
- sub 20min



Python is easy

In [3]:
raise Exception('An error')

Exception: An error

# How to capture an exception?

## Quite simple use `except`

```python
 try:
 ...
 except KeyError:
    print('Got you')
```

## You can capture multiple type of exceptions

```python
 try:
   # request('http://api.wrnch.ai/login')     
 except BadRequest:
   # complain to the user     
 except (Exception, ValueError):
   # Maybe thats bad    
```

## Watchout the interpreter won't save you

```python
 try:
    # request('http://api.wrnch.ai/login')
 except a_variable:
    pass
 except foo():
    pass
 except KyeError:
    pass
```

# What else? finally?

In [53]:
def foo():
    try:
        print(1)
    except:
        print(2)
    else:
        print(3)
    finally:
        print(4)
foo()

1
3
4


# What if we use return?

In [55]:
def foo():
    try:
        return(1)
    except:
        return(2)
    else:
        return(3)
    finally:
        return(4)
print(foo())

4


# How about logging in except?

In [15]:
try:
    val = 10/0
except Exception:
    logging.error('There was an error')

ERROR:root:There was an error


In [16]:
try:
    val = 10/0
except Exception:
    logging.error('There was an error', exc_info=True)

ERROR:root:There was an error
Traceback (most recent call last):
  File "<ipython-input-16-67c406685699>", line 2, in <module>
    val = 10/0
ZeroDivisionError: division by zero


# You can also use logging.exception

In [19]:
try:
    val = 10/0
except Exception:
    logging.exception('There was an error')

ERROR:root:There was an error
Traceback (most recent call last):
  File "<ipython-input-19-2f68e75b7684>", line 2, in <module>
    val = 10/0
ZeroDivisionError: division by zero


# How about logging the exception only?

In [56]:
try:
    raise KeyError()
except Exception as e:
    logging.error(f'{e}')

ERROR:root:


In [28]:
try:
    raise KeyError()
except Exception as e:
    logging.error(f'There was an error: {repr(e)}')

ERROR:root:There was an error: KeyError()


Use the represention to make sure you dont have an empty message

# Raise within an except

In [31]:
try:
    10/0
except Exception:
    raise Exception('Bad code, could not handle it')

Exception: Bad code, could not handle it

In [32]:
try:
    10/0
except Exception:
    print('Doing some prework')
    raise

Doing some prework


ZeroDivisionError: division by zero

In [34]:
try:
    10/0
except Exception as e:
    print('Ok storing for later')
    ee = e
raise ee

Ok storing for later


ZeroDivisionError: division by zero

# Chaining exceptions

In [37]:
try:
    10/0
except Exception as e:
 raise Exception('Why does it even divide by 0?') from e

Exception: Why does it even divide by 0?

# Eliding the handled exception

In [38]:
try:
    10/0
except Exception as e:
 raise Exception('Too bad contact support') from None

Exception: Too bad contact support

No trace of the previous exception

# Create custom exception

The Exception hierarchy in py38

![exception hierarchy](./hierarchy.png)

ref: https://docs.python.org/3.8/library/exceptions.html#exception-hierarchy


# Exceptions attributes

In [23]:
dic = {'hello':'world'}
try:
    print(dic['world'])
except Exception as err:
 exc = err

In [24]:
exc.message

AttributeError: 'KeyError' object has no attribute 'message'

In [28]:
dic = {'hello':'world'}
try:
    print(dic['world'])
except Exception as err:
 exc = err

In [26]:
print(f'the error is {exc}')

the error is 'world'


In [22]:
print(f'the errorb is {repr(exc)}')
print(f'the exception has this argument: {exc.args}')

the error is KeyError('world')
the exception has this argument: ('world',)


# Creating new classes

In [29]:
# Don't
class AnError(BaseException):
    pass

In [31]:
# Do 
class AnError(Exception):
    pass

In [41]:
e = AnError('hello','world','!')
print(f'this happened: {str(e)}')
print(f'No actually this: {repr(e)}')
print(f'with those args: {e.args}')

this happened: ('hello', 'world', '!')
No actually this: AnError('hello', 'world', '!')
with those args: ('hello', 'world', '!')


# Create your own hierarchy

In [47]:
class HTTPException(Exception):
    pass

class BadRequest(HTTPException, ValueError):
    pass

# ...


## Conclusions

Now you should handle exception this way

In [50]:
try:
    10/0
except (ZeroDivisionError, ValueError):
    logging.warning('Something bad certainly happened', exc_info=True)
    raise BadRequest('Dear user, review your inputs') from None

Traceback (most recent call last):
  File "<ipython-input-50-8354ecde736b>", line 2, in <module>
    10/0
ZeroDivisionError: division by zero


BadRequest: Dear user, review your inputs